# WP4: Climatology and Bias - Near Surface Air Temperature

## Import libraries

Switch warnings off for better readability.\
Set matplotlib default settings.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-talk")

warnings.filterwarnings("ignore")

## Define time period

In [ ]:
year_start = 1985
year_stop = 1987

common_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

## Define requests

In [ ]:
request_era = (
    "reanalysis-era5-single-levels-monthly-means",
    {
        "product_type": "monthly_averaged_reanalysis",
        "format": "netcdf",
        "time": "00:00",
        "variable": "2m_temperature",
        **common_request,
    },
)

request_sim = (
    "projections-cmip6",
    {
        "format": "zip",
        "temporal_resolution": "monthly",
        "experiment": "historical",
        "variable": "near_surface_air_temperature",
        "model": "cmcc_cm2_sr5",
        **common_request,
    },
)

## Define functions

In [ ]:
def resample_and_regrid_sim(ds, grid_out):
    ds_seasonal = diagnostics.seasonal_weighted_mean(ds)
    return diagnostics.regrid(ds_seasonal, grid_out, method="bilinear", periodic=True)


def kelvin_to_celsius(da):
    """Convert from kelvin to celsius."""
    with xr.set_options(keep_attrs=True):
        da -= 273.15
    da.attrs["units"] = "°C"
    return da

## Download data and interpolate

In [ ]:
chunks = {"year": 1}

ds_era_seasonal = download.download_and_transform(
    *request_era,
    chunks=chunks,
    transform_func=diagnostics.seasonal_weighted_mean,
    transform_chunks=False,
)

ds_sim_annual = download.download_and_transform(
    *request_sim,
    chunks=chunks,
    transform_func=diagnostics.annual_weighted_mean,
    transform_chunks=False,
)

ds_reg_seasonal = download.download_and_transform(
    *request_sim,
    chunks=chunks,
    transform_func=resample_and_regrid_sim,
    transform_func_kwargs={
        "grid_out": ds_era_seasonal[["longitude", "latitude"]],
    },
    transform_chunks=False,
)

## Compute annual mean and seasonal bias

In [ ]:
# Convert to celsius
da_sim_annual = kelvin_to_celsius(ds_sim_annual["tas"])

# Compute bias
with xr.set_options(keep_attrs=True):
    da_bias = ds_reg_seasonal["tas"] - ds_era_seasonal["t2m"]
da_bias.attrs["long_name"] += " Bias"

## Define labels and settings

In [ ]:
# labels
info = {key: request_sim[1][key] for key in ("model", "experiment", "variable")}
info["period"] = f"{request_sim[1]['year'][0]}-{request_sim[1]['year'][-1]}"
fig_prefix = "_".join(info.values())
suptitle = "; ".join([f"{k.title()}: {v}" for k, v in info.items()])

# matplotlib settings
savefig_kwargs = {"bbox_inches": "tight"}
suptitle_kwargs = {"x": 0.5, "y": 1, "ha": "center", "va": "bottom"}

## Plot and save annual mean

In [ ]:
p1 = plot.projected_map(da_sim_annual, levels=range(-30, 31, 5), cmap="YlOrRd")
p1.figure.suptitle(suptitle, **suptitle_kwargs)
p1.figure.savefig(f"{fig_prefix}_annual.png", **savefig_kwargs)

## Plot and save seasonal bias

In [ ]:
p2 = plot.projected_map(
    da_bias, levels=range(-6, 7), cmap="RdBu_r", col="season", col_wrap=2
)
p2.fig.suptitle(suptitle, **suptitle_kwargs)
p2.fig.savefig(f"{fig_prefix}_seasonal_bias.png", **savefig_kwargs)